Scenario:
You're building an AI Assistant that:

1. Writes a product pitch based on input.
2. Shares it with the user for feedback.
3. If feedback is not "good", the LLM will rewrite the pitch again.
4. Loop continues until user says "good".
5. Then, it ends the graph.

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
from typing import Optional

In [2]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

State Model

In [3]:
class PitchState(BaseModel):
    topic: str
    pitch: Optional[str] = None
    feedback: Optional[str] = None    

Nodes creation

In [9]:
# Get pitch Node
def get_pitch_node(state:PitchState):
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY)
    prompt =  f"Write a short product pitch about: {state.topic}"
    result = llm.invoke(prompt)
    
    updated = state.model_dump()
    updated['pitch'] = result.content
    return PitchState(**updated)

# Get feedback Node
def get_feedback_node(state:PitchState):
    # Simulate user giving feedback
    # In real-world this could be input() or a UI form
    print("\n📝 PITCH:\n", state.pitch)
    feedback = input("Do you like this pitch? Type 'good' to accept or 'no' to regenerate: ").strip().lower()
    
    updated = state.model_dump()
    updated['feedback'] = feedback
    return PitchState(**updated)

# Condition: Loop until feedback is 'good'

def check_feedback(state:PitchState):
    if state.feedback == "good":
        return END
    return "get_pitch"

Graph Structure

In [10]:
from langgraph.graph import StateGraph, END

graph = StateGraph(state_schema=PitchState)
graph.add_node("get_pitch",get_pitch_node)
graph.add_node("get_feedback", get_feedback_node)

graph.set_entry_point("get_pitch")
graph.add_edge("get_pitch", "get_feedback")
graph.add_conditional_edges("get_feedback",check_feedback,{
    END:END,
    "get_pitch":"get_pitch"
})

app = graph.compile()

In [12]:
initial_state = PitchState(topic="AI-powered workout tracker")
result = app.invoke(initial_state)
print("\n✅ Final Pitch:\n", result['pitch'])



📝 PITCH:
 ## Tired of the Same Old Workout Routine? Meet AI-Powered Fitness!

**Stop guessing, start achieving!** Introducing the revolutionary workout tracker that learns *you*. Our AI-powered app analyzes your performance, goals, and even your biometrics to create a personalized workout plan that evolves with you.

**Here's what sets us apart:**

* **Personalized Plans:** Ditch the generic routines. Our AI crafts workouts tailored to your specific needs and fitness level.
* **Smart Progress Tracking:** We don't just count reps. We analyze your form, intensity, and recovery to provide actionable insights.
* **Real-time Feedback:** Get instant guidance on your technique to prevent injuries and maximize your results.
* **Motivation, Reimagined:** Our AI provides personalized encouragement and adjusts your plan based on your progress, keeping you engaged and motivated.

**Stop wasting time and effort on workouts that don't work. Download our AI-powered fitness tracker today and unlock y